In [15]:
import json
import urllib.request

import requests
import os
import env

# url = "http://metabase.jabama.com/api/card/" + str(env.ACCOMMODATION_CARD_ID) + "/query"

# headers = {
#     'X-Metabase-Session': "daabf2cf-c6f5-413f-9cb3-e1bffad2d699",
#     'Cache-Control': "no-cache",
#     'Postman-Token': "4914830f-7ad3-4213-8738-1133e80d1c9b"
#     }

# response = requests.request("POST", url, headers=headers)

# cols = json.loads(response.text)['data']['cols']
# rows = json.loads(response.text)['data']['rows']

# col_titles = []
# for col in cols:
#     col_titles.append(col['name'])
 
# #Formatting the message content and storing it in a plain text
# plain_text = ""

# col_titles.pop(0)
# for row in rows:
#     indx = 1
#     for col in col_titles:
#         plain_text = plain_text + str(row[0]) + ", " + str(col) + ": " + row[indx] + os.linesep
#         indx = indx + 1

# payload = {
#     "channel" : "UR6RJUHGX",
#     "text" : plain_text,
#     "as_user": True
# }

# headers = {
#     'Content-Type': "application/json",
#     'Authorization': "Bearer " + env.METABOT_TOKEN,
#     'Cache-Control': "no-cache",
#     'Postman-Token': "3dea07a8-cf24-47cb-b8cd-2a141a41a0b3"
#     }

# response = requests.request("POST", SLACK_API_POST_MESSAGE, data=str(payload), headers=headers)

def getMetaCard(card_id):

    url = "http://metabase.jabama.com/api/card/" + card_id + "/query"

    headers = {
        'X-Metabase-Session': "daabf2cf-c6f5-413f-9cb3-e1bffad2d699",
        'Cache-Control': "no-cache",
        'Postman-Token': "4914830f-7ad3-4213-8738-1133e80d1c9b"
        }

    response = requests.request("POST", url, headers=headers)
    return response

def getDataFromResponse(response):
    cols = json.loads(response.text)['data']['cols']
    rows = json.loads(response.text)['data']['rows']

    col_titles = []
    for col in cols:
        col_titles.append(col['name'])
        
    return {
        "columns": col_titles,
        "rows": rows
    }

def formatTheMessageText(responseTable):
    plain_text = ""

    responseTable.get("columns").pop(0)
    for row in responseTable.get("rows"):
        indx = 1
        for col in responseTable.get("columns"):
            plain_text = plain_text + str(row[0]) + ", " + str(col) + ": " + str(row[indx]) + os.linesep
            indx = indx + 1
    return plain_text

def getUserIDByEmail(email):

    querystring = {
        "email" : email
    }

    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer " + env.METABOT_TOKEN,
        'User-Agent': "PostmanRuntime/7.20.1",
        'Accept': "*/*",
        'Cache-Control': "no-cache",
        'Postman-Token': "1eed9ef6-4ab9-44d9-bf2a-650df7574c84,7e6fc185-e1fd-4ca8-9806-782d0de54a84",
        'Host': "slack.com",
        'Accept-Encoding': "gzip, deflate",
        'Connection': "keep-alive",
        'cache-control': "no-cache"
        }

    response = requests.request("GET", env.SLACK_API_LOOK_UP_BY_EMAIL, headers=headers, params=querystring)
    return json.loads(response.text)['user']['id']
        
def sendMessageToSlackUser(user_id, message_text):
    payload = {
        "channel" : user_id,
        "text" : message_text,
        "as_user": True
    }
    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer " + env.METABOT_TOKEN,
        'Cache-Control': "no-cache",
        'Postman-Token': "3dea07a8-cf24-47cb-b8cd-2a141a41a0b3"
        }

    response = requests.request("POST", env.SLACK_API_POST_MESSAGE, data=str(payload), headers=headers)
    
    return json.loads(response.text)['ok']

metacard_response = getMetaCard(env.CARD_ID_ACCOMMODATION)
response_data = getDataFromResponse(metacard_response)
message_text = formatTheMessageText(response_data)

user_id = getUserIDByEmail(env.)
send_message_status = sendMessageToSlackUser(user_id, message_text)

print(send_message_status)

True
